In [50]:
import sqlite3
import pandas
import av_getdata
import credentials
from pymongo import MongoClient

In [3]:
conn = MongoClient('localhost', credentials.db_port)

In [4]:
db = conn.stocksdata.usa_stocks

In [6]:
c = db.find_one({'ticker': 'IBM'},{'data': {
                'date':1,
                'open':1,
                'high':1,
                'low':1,
                'close':1,
                'volume':1
                }
                })
c

{'_id': ObjectId('63c6d286579e023218b8432c'),
 'data': {'date': [datetime.datetime(2023, 1, 13, 0, 0),
   datetime.datetime(2023, 1, 12, 0, 0),
   datetime.datetime(2023, 1, 11, 0, 0),
   datetime.datetime(2023, 1, 10, 0, 0),
   datetime.datetime(2023, 1, 9, 0, 0),
   datetime.datetime(2023, 1, 6, 0, 0),
   datetime.datetime(2023, 1, 5, 0, 0),
   datetime.datetime(2023, 1, 4, 0, 0),
   datetime.datetime(2023, 1, 3, 0, 0),
   datetime.datetime(2022, 12, 30, 0, 0),
   datetime.datetime(2022, 12, 29, 0, 0),
   datetime.datetime(2022, 12, 28, 0, 0),
   datetime.datetime(2022, 12, 27, 0, 0),
   datetime.datetime(2022, 12, 23, 0, 0),
   datetime.datetime(2022, 12, 22, 0, 0),
   datetime.datetime(2022, 12, 21, 0, 0),
   datetime.datetime(2022, 12, 20, 0, 0),
   datetime.datetime(2022, 12, 19, 0, 0),
   datetime.datetime(2022, 12, 16, 0, 0),
   datetime.datetime(2022, 12, 15, 0, 0),
   datetime.datetime(2022, 12, 14, 0, 0),
   datetime.datetime(2022, 12, 13, 0, 0),
   datetime.datetime(2022, 1

In [22]:
def update(db,ticker,date,data):
    open,high,low,close,adj,vol = data[0],data[1],data[2],data[3],data[4],data[5]
    db.update_one(
        {'ticker': ticker}, 
        {'$push': {
            'data.date' : date,
            'data.open' : open,
            'data.high' : high,
            'data.low' : low,
            'data.close' : close,
            'data.adjusted_close' : adj,
            'data.volume' : vol
            }
        }
    )

In [15]:
update(db,'IBM',)

In [16]:
import pandas
from av_getdata import *
from pprint import pprint
import sys

In [24]:
data = StocksData()
ibm,meta = data.daily_adjusted('IBM','full')

In [18]:
i = ibm

In [25]:
for d,values in zip(ibm.index,ibm.values):
    date = d
    data = values[:-2] # no dividend amount, split coefficient
    update(db,'IBM',date,data)
    

In [39]:
ibm_date = db.find_one({'ticker': 'IBM'}, {'data': {
    'date':1,
}})

In [58]:
ibm_data = db.find_one({'ticker': 'IBM'}, {'data': {
    'date':1,
    'open':1,
    'high':1,
    'low':1,
    'close':1,
    'volume':1
}})

In [5]:
#ibm_data['data']['date']
ibm_data['data']['volume']

NameError: name 'ibm_data' is not defined

In [54]:
ibm.iloc[66].name in ibm_data['data']['date']

True

In [2]:
import database
conn = database.DatabaseConnection()
data = conn.new_entry('KO')

In [4]:
data.index

RangeIndex(start=0, stop=5839, step=1)

In [10]:
import datetime
datetime.datetime.today()

datetime.datetime(2023, 1, 24, 11, 56, 37, 776513)

In [22]:
newdata = conn.fetch('KO')
newdata

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2023-01-23,60.280,60.565,60.0500,60.23,60.230000,12613428.0,0.0,1.0
2023-01-20,59.720,60.090,59.4000,60.08,60.080000,14237702.0,0.0,1.0
2023-01-19,59.725,60.640,59.7050,59.72,59.720000,14894455.0,0.0,1.0
2023-01-18,61.510,61.590,59.6817,59.81,59.810000,19679436.0,0.0,1.0
2023-01-17,61.730,62.380,61.5600,61.68,61.680000,18327779.0,0.0,1.0
...,...,...,...,...,...,...,...,...
1999-11-05,57.940,58.440,57.6300,58.13,15.450016,4104800.0,0.0,1.0
1999-11-04,57.500,57.500,56.0600,56.69,15.067287,3768500.0,0.0,1.0
1999-11-03,57.190,57.750,56.0600,56.88,15.117786,3958700.0,0.0,1.0


In [26]:
newdata = newdata.drop(['7. dividend amount', '8. split coefficient'],axis=1)

In [38]:
newdata = newdata.sort_index()

In [39]:
old_IBM = conn._data('IBM')
old_IBM = conn.make_dataframe(old_IBM)
old_IBM = old_IBM.set_index('date')
old_IBM = old_IBM.sort_index()

In [45]:
newdata.columns = ['open', 'high', 'low', 'close', 'adjusted close', 'volume']

In [87]:
newdata = newdata.drop(['adjusted close'], axis=1)

KeyError: "['adjusted close'] not found in axis"

In [88]:
newdata

,open,high,low,close,volume
date,,,,,
1999-11-01,58.250,59.380,57.8800,59.00,4212600.0
1999-11-02,58.630,58.940,57.3100,57.75,3226800.0
1999-11-03,57.190,57.750,56.0600,56.88,3958700.0
1999-11-04,57.500,57.500,56.0600,56.69,3768500.0
1999-11-05,57.940,58.440,57.6300,58.13,4104800.0
...,...,...,...,...,...
2023-01-17,61.730,62.380,61.5600,61.68,18327779.0
2023-01-18,61.510,61.590,59.6817,59.81,19679436.0
2023-01-19,59.725,60.640,59.7050,59.72,14894455.0


In [89]:
old_IBM

,open,high,low,close,volume
date,,,,,
1999-11-01,98.50,98.81,96.37,96.75,9551800.0
1999-11-02,96.75,96.81,93.69,94.81,11105400.0
1999-11-03,95.87,95.94,93.50,94.37,10369100.0
1999-11-04,94.44,94.44,90.00,91.56,16697600.0
1999-11-05,92.75,92.94,90.19,90.25,13737600.0
...,...,...,...,...,...
2023-01-09,144.08,145.47,143.40,143.55,3987782.0
2023-01-10,143.61,144.85,142.90,144.80,2152172.0
2023-01-11,145.00,145.53,143.45,145.26,3268738.0


In [81]:
testdf = pandas.concat([old_IBM,newdata],ignore_index=False)

In [85]:
test = testdf[~testdf.index.duplicated(keep='first')]

In [86]:
test

,open,high,low,close,volume
date,,,,,
1999-11-01,98.500,98.810,96.3700,96.75,9551800.0
1999-11-02,96.750,96.810,93.6900,94.81,11105400.0
1999-11-03,95.870,95.940,93.5000,94.37,10369100.0
1999-11-04,94.440,94.440,90.0000,91.56,16697600.0
1999-11-05,92.750,92.940,90.1900,90.25,13737600.0
...,...,...,...,...,...
2023-01-17,61.730,62.380,61.5600,61.68,18327779.0
2023-01-18,61.510,61.590,59.6817,59.81,19679436.0
2023-01-19,59.725,60.640,59.7050,59.72,14894455.0
